# MXNet Model Server

This Notebook is borrowed from: https://github.com/TalkAI/facial-emotion-recognition-gluon/blob/master/notebooks/Gluon_FERPlus.ipynb

## Install the required Packages locally

We will need the PyPi packages listed below to test model server locally, and to perform image pre-processing prior to the model inference.

In [ ]:
!pip install -q mxnet-model-server
print('mxnet-model-server installed')
!pip install -q scikit-image==0.13.0
print('scikit-learn installed')
!pip install -q opencv-python
print('opencv-python installed')

In [1]:
# We define the model's input and output type and shape via signature.json
!cat ./model_archive_resources/signature.json

{
  "input_type": "image/jpeg", 
  "inputs": [
    {
      "data_shape": [1, 1, 64, 64], 
      "data_name": "data"
    }
  ], 
  "outputs": [
    {
      "data_shape": [1, 8],
      "data_name": "hybridsequential0_dense2_fwd"
    }
  ], 
  "output_type": "application/json"
}

In [2]:
# We define the model's class label names via synset.txt
!cat ./model_archive_resources/synset.txt

neutral
happiness
surprise
sadness
anger
disgust
fear
contempt

In [3]:
# And lastly, we define custom code for request handling via python code other auxiliary files
!cat ./model_archive_resources/fer_service.py

import numpy as np
from mms.utils.mxnet import image
from mms.model_service.mxnet_model_service import MXNetBaseService
from skimage import transform
import mxnet as mx
import cv2 as cv

# One time initialization of Haar Cascade Classifier to extract and crop out face
face_detector = cv.CascadeClassifier('haarcascade_frontalface.xml')
# Classifier parameter specifying how much the image size is reduced at each image scale
scale_factor = 1.3
# Classifier parameter how many neighbors each candidate rectangle should have to retain it
min_neighbors = 5

def crop_face(image):
    """Attempts to identify a face in the input image.

    Parameters
    ----------
    image : array representing a BGR image

    Returns
    -------
    array
        The cropped face, transformed to grayscale. If no face found returns None

    """
    gray_image = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
    face_roi_list = face_detector.detectMultiScale(gray_image, scale_factor, min_neighbors)
    
    if (len(fac

## Download Model

In [ ]:
%cd model_archive_resources
!curl -O https://s3.amazonaws.com/mxnet-demo-models/models/fer/gluon_ferplus-0000.params
!curl -O https://s3.amazonaws.com/mxnet-demo-models/models/fer/gluon_ferplus-symbol.json

## Export model

In [4]:
%cd ../
!pwd

/Users/wamy/nswamy/deepengine/workspace/dl-introduction-apache-mxnet-odsc-2018
/Users/wamy/nswamy/deepengine/workspace/dl-introduction-apache-mxnet-odsc-2018


In [ ]:
# Let's package everything up into a Model Archive bundle
!mxnet-model-export --model-name ferplus \
--service-file-path ./model_archive_resources/fer_service.py \
--model-path ./model_archive_resources

In [7]:
%cd part3_model_server/model_archive_resources

/Users/wamy/nswamy/deepengine/workspace/dl-introduction-apache-mxnet-odsc-2018/part3_model_server/model_archive_resources


In [8]:
!ls -l ferplus.model

-rw-r--r--  1 wamy  1896053708  35984920 Oct 30 00:03 ferplus.model


In [19]:
# Spawning a new process to run the server
import subprocess as sp
server = sp.Popen("mxnet-model-server --models ferplus=ferplus.model", shell=True)

In [20]:
# Check out the health endpoint
!curl http://127.0.0.1:8080/ping

{"health":"healthy!"}


In [21]:
!ls

fer_service.py              mms_app.log
ferplus                     signature.json
ferplus.model               synset.txt
haarcascade_frontalface.xml


In [24]:
!curl -X POST http://127.0.0.1:8080/ferplus/predict -F "data=@../happy.jpg"

{"prediction":[{"neutral":0.3882121741771698},{"happiness":0.5335745811462402},{"surprise":0.0024918382987380028},{"sadness":0.05849442631006241},{"anger":0.008850693702697754},{"disgust":0.0009525333880446851},{"fear":0.0007643798599019647},{"contempt":0.006659360136836767}]}


In [15]:
# Lastly, we'll terminate the server
server.terminate()